In [5]:
import sklearn
import re
from gensim.models import Word2Vec
import numpy as np
import gensim

# import /media/shanmukh/datasets/rallaakhil/skip-thoughts

In [6]:
path = "../../Medical-Text-Simplification./Dataset/sentence-aligned.v2/complex-simple.txt"
f =open(path,"r")

In [7]:
def strip_punct(x):
    return re.sub(r"[!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~\n\t–\\]"," ",x).lower()
def strip_non_ascii(x):
    return re.sub(r'[^\x00-\x7F]',' ',x) 

In [8]:
simple = []
compl = []
sentences = []
while True:
    try:
        sent = f.readline().split("\t")
    except:
        break
#     print (sent)
    if len(sent) == 1:
        break
    sentences.append(strip_non_ascii(strip_punct(sent[0])).split())
    sentences.append(strip_non_ascii(strip_punct(sent[1][:-1])).split())

    if sent[0]==sent[1][:-1]:
            
        simple.append(strip_non_ascii(strip_punct(sent[0])).split())
        compl.append(strip_non_ascii(strip_punct(sent[1][:-1])).split())
    else:
        continue
        
        
# model = Word2Vec(sentences, min_count=2,workers=4,size=25)
# model = gensim.models.KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)
# len(words)

In [9]:
sentences
# model = gensim.models.word2vec(sentences, size=200, workers=10, window=111, min_count=1, max_vocab_size=None, sg=1, hs=1)

[['it', 'is', 'the', 'county', 'seat', 'of', 'alfalfa', 'county'],
 ['it', 'is', 'the', 'county', 'seat', 'of', 'alfalfa', 'county'],
 ['cherokee',
  'is',
  'a',
  'city',
  'of',
  'oklahoma',
  'in',
  'the',
  'united',
  'states'],
 ['cherokee',
  'is',
  'a',
  'city',
  'in',
  'alfalfa',
  'county',
  'oklahoma',
  'united',
  'states'],
 ['skateboard',
  'decks',
  'are',
  'normally',
  'between',
  '28',
  'and',
  '33',
  'inches',
  'long'],
 ['skateboard',
  'decks',
  'are',
  'usually',
  'between',
  '28',
  'and',
  '33',
  'inches',
  'long'],
 ['the',
  'bottom',
  'of',
  'the',
  'deck',
  'can',
  'be',
  'printed',
  'with',
  'a',
  'design',
  'by',
  'the',
  'maker',
  'or',
  'it',
  'can',
  'be',
  'blank'],
 ['the',
  'underside',
  'of',
  'the',
  'deck',
  'can',
  'be',
  'printed',
  'with',
  'a',
  'design',
  'by',
  'the',
  'manufacturer',
  'blank',
  'or',
  'decorated',
  'by',
  'any',
  'other',
  'means'],
 ['the',
  'longboard',
  'was',

In [10]:
model = gensim.models.Word2Vec(sentences, size=200, workers=10, window=111, min_count=1, max_vocab_size=None, sg=1, hs=1)

In [11]:
lens = []
model.save("w2v.bin")
for i in range(len(simple)):
    lens.append(len(simple[i]))
    lens.append(len(compl[i]))

In [7]:
max_len = max(lens)
print (max_len)

141


In [8]:
print(model)
# summarize vocabulary
# words = list(model.wv.vocab)
# print(words)
# access vector for one word
# print(model['sentence'])
print((np.asarray(model['sentence']-model["sentences"])**2).sum())
# print(model["sentences"])
# print(model["cat"])
print((np.asarray(model['india']-model["pakistan"])**2).sum())
len_vec = len(model["cat"])
# pritn
print (len(simple))

4.825439
8.876465
50352


In [9]:
X = []
Y = np.ones((2*len(simple),1))
for i in range(len(simple)):
    temp_x = np.zeros((len_vec))
    Y[i] = 0
    for j in range(len(simple[i])):
        try:
            temp_x[:] += np.asarray(model[simple[i][j]])
        except:
            continue   
    X.append(temp_x)
    
for i in range(len(compl)):
    temp_x = np.zeros((len_vec))
    for j in range(len(simple[i])):
        try:
            temp_x[:] += np.asarray(model[compl[i][j]])
        except:
            continue
    X.append(temp_x)

In [10]:
# X = np.asarray(X)
# print (len(X))
# for i in range(25*max_len):
#     X[:,i] = (X[:,i]-np.mean(X[:,i]))/np.var(X[:,i]) 

In [11]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100,whiten=True)
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=True)

In [12]:
X_ = pca.transform(X)


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2)
clf = RandomForestClassifier()
clf.fit(x_train,y_train)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
clf.score(x_test,y_test)

0.11861377290104762

In [ ]:
X[0]

In [20]:
# len(X)
print (X[1])

[ 1.25122070e-01  7.60628700e-01  9.10659790e-01  6.24145508e-01
 -1.24328613e-01 -1.56182861e+00 -2.44318008e-01 -8.97399902e-01
  1.06843758e+00  9.64965820e-01  1.32141113e-01 -1.48922729e+00
 -8.92944336e-02  1.70898438e-01 -9.82204437e-01  4.99084473e-01
  6.11999512e-01 -1.07910156e-01 -1.46789551e-01  3.44207764e-01
 -1.14086914e+00  1.13098145e+00 -3.33251953e-01 -6.82815552e-01
 -6.17370605e-02 -4.22698975e-01 -5.62683105e-01  1.67919159e+00
  6.71371460e-01  5.43579102e-01 -1.74331665e-01 -3.39805603e-01
 -4.99328613e-01  1.34181976e-01  4.13696289e-01  2.47802734e-01
  2.71606445e-01  2.18765259e-01  3.91967773e-01  1.35540009e-01
  2.63885498e-01 -9.39880371e-01  1.34631348e+00 -5.69595337e-01
  1.04914856e+00 -3.26065063e-01 -6.07513428e-01 -3.24363708e-01
  3.55529785e-01 -2.79541016e-02 -6.15844727e-01  2.93334961e-01
  1.35046387e+00  1.42623901e-01  3.33496094e-01  9.30541992e-01
 -2.19146729e-01 -5.58624268e-02  4.41345215e-01 -4.35546875e-01
 -7.17132568e-01  8.14300